In [ ]:
import json
import time
import re
import random
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch
import math

# ANSI color codes for terminal output
BLUE = '\033[94m'     # For sample information
GREEN = '\033[92m'    # For correct predictions and success messages
RED = '\033[91m'      # For incorrect predictions
YELLOW = '\033[93m'   # For predictions and headers
CYAN = '\033[96m'     # For progress information
PURPLE = '\033[95m'   # For model responses
BOLD = '\033[1m'      # Bold text
ENDC = '\033[0m'      # End color

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset Loading Functions

Functions to load different dataset formats (JSON and JSONL)

In [ ]:
def load_dataset(file_path, dataset_name, limit=150):
    """Load samples from various datasets with appropriate format handling"""
    print(f"{CYAN}Loading {dataset_name} dataset from {file_path}...{ENDC}")
    data = []

    if not os.path.exists(file_path):
        print(f"{RED}File not found: {file_path}{ENDC}")
        return []

    if file_path.endswith('.jsonl'):
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():  # Skip empty lines
                    data.append(json.loads(line))
                    if len(data) >= limit:
                        break
    else:  # .json files
        with open(file_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
            if isinstance(json_data, list):
                data = json_data[:limit]
            else:
                # Handle nested structures if needed
                if 'data' in json_data:
                    data = json_data['data'][:limit]
                else:
                    print(f"{YELLOW}Warning: Unexpected JSON structure in {file_path}{ENDC}")
                    data = [json_data]  # Just use the whole object as one sample

    print(f"{CYAN}Loaded {len(data)} samples from {dataset_name}.{ENDC}")
    return data

## Prompt Creation Functions

Different prompting strategies for each dataset type

In [ ]:
def create_prompt(sample, dataset_name):
    """Create appropriate prompts based on dataset type"""

    if dataset_name == "VitaminC" or dataset_name == "FEVER" or dataset_name == "FEVEROUS":
        # Fact verification datasets
        if dataset_name == "VitaminC":
            claim = sample["claim"]
            evidence = sample["evidence"]
        elif dataset_name == "FEVER":
            claim = sample["claim"]
            evidence = sample.get("evidence", "") or sample.get("context", "")
        elif dataset_name == "FEVEROUS":
            claim = sample["claim"]
            evidence = sample.get("evidence", "") or sample.get("context", "")

        return create_fact_verification_prompt(claim, evidence)

    elif dataset_name == "HotpotQA" or dataset_name == "2WikiMultihopQA":
        # Multi-hop QA datasets
        question = sample.get("question", "") or sample.get("query", "")
        context = sample.get("context", "")
        if not context and "original_context" in sample:
            context = sample["original_context"]

        return create_qa_prompt(question, context)

    elif dataset_name == "SVAMP":
        # Math word problem dataset
        question = sample.get("question", "") or sample.get("body", "")
        return create_math_prompt(question)

    elif dataset_name == "Bamboogle":
        # Bamboogle dataset - assume it's a QA task
        question = sample.get("question", "")
        context = sample.get("context", "")

        if not question and "answer" in sample:
            # If no question is provided but there's an answer, create a generic prompt
            return

        return create_qa_prompt(question, context)

    else:
        # Generic prompt for unknown datasets
        return f"Please analyze this data and think step-by-step:\n\n{json.dumps(sample, indent=2)}\n\nAfter your thinking, provide your answer by writing 'ANSWER: <finalAnswer>' on the same line."

def create_fact_verification_prompt(claim, evidence):
    """Create a prompt for fact checking"""
    prompt = f"""Claim: {claim}

Evidence: {evidence}

Think step-by-step to determine if the evidence SUPPORTS, REFUTES, or provides NOT ENOUGH INFO for the claim.

After your thinking, end your answer with one of these:
ANSWER: SUPPORTS or ANSWER: REFUTES or ANSWER: NOT ENOUGH INFO

Always end with the final answer."""
    return prompt

def create_qa_prompt(question, context):
    """Create a prompt for question answering tasks"""
    prompt = f"""

Question: {question}

Think step-by-step to answer the question based on the context.

After your thinking, provide your answer in this exact format:
ANSWER: <finalAnswer>

Always end with the final answer."""
    return prompt

def create_math_prompt(question):
    """Create a prompt for math word problems"""
    prompt = f"""Problem: {question}

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>
Never end your final answer with a latex boxed format.
Always end with the final answer."""
    return prompt


## Model Inference Function

In [ ]:
def run_inference(model, tokenizer, prompt, device="cuda"):
    """Run inference on the model"""
    # Tokenize input with attention mask
    encoding = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128000)

    inputs = {
        'input_ids': encoding.input_ids.to(device),
        'attention_mask': encoding.attention_mask.to(device)
    }

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=2048,  # Adjust based on desired output length
            temperature=0.2,      # Lower temperature for focused output
            do_sample=True,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


## Prediction Extraction Functions

Functions to extract structured predictions from model outputs

In [ ]:
import re

def extract_prediction(output, dataset_name, sample=None):
    """Extract the final answer from the model's output.
    1) Tries to find lines starting with 'ANSWER:' or 'The final answer is:' and extracts the last one.
    2) If none found, falls back to dataset-specific extraction.
    """
    pattern = r'(?:^|\n)(?:ANSWER:|The final answer is:)\s*(.*)$'
    matches = re.findall(pattern, output, flags=re.MULTILINE)

    if matches:
        last_answer = matches[-1].strip()
        if dataset_name == "SVAMP":
            parsed = _handle_math_answer(last_answer, output)
            return parsed
        return last_answer

    # If no direct lines found, fall back to dataset-specific extraction.
    extraction_methods = {
        "VitaminC": extract_fact_verification_prediction,
        "FEVER": extract_fact_verification_prediction,
        "FEVEROUS": extract_fact_verification_prediction,
        "HotpotQA": extract_qa_prediction,
        "2WikiMultihopQA": extract_qa_prediction,
        "SVAMP": extract_math_prediction,  # fallback if we can't find any line above
        "Bamboogle": extract_qa_prediction,
    }

    # Attempt dataset-specific extraction.
    return extraction_methods.get(dataset_name, lambda x: x.strip())(output)


def extract_fact_verification_prediction(output):
    """Extract fact verification label from model output."""
    output_lower = output.lower()

    # 1) Check if there's an ANSWER: line with a direct label.
    answers = re.findall(r'ANSWER:\s*(SUPPORTS|REFUTES|NOT ENOUGH INFO)', output_lower, re.MULTILINE)
    if answers:
        return answers[-1]

    # 2) Look for known patterns in the text.
    verdict_patterns = [
        r"verdict\s*:?\s*(supports|refutes|not enough info)",
        r"therefore,?\s+(the evidence)?\s*(supports|refutes|provides not enough info)",
        r"(my conclusion|my answer) is\s*:?\s*(supports|refutes|not enough info)",
        r"the evidence (supports|refutes|provides not enough info)",
        r"(supports|refutes|not enough info) the claim"
    ]
    for pattern in verdict_patterns:
        matches = re.findall(pattern, output_lower)
        if matches:
            label = matches[-1][-1] if isinstance(matches[-1], tuple) else matches[-1]
            return _normalize_fact_label(label)

    # 3) Try searching the last quarter or entire text.
    last_quarter = output_lower[3*len(output_lower)//4:]
    return _search_label(last_quarter, output_lower)


def extract_qa_prediction(output):
    """Extract the answer from QA model output."""
    answers = re.findall(r'^ANSWER:\s*(.*?)$', output, re.MULTILINE)
    if answers:
        return answers[-1].strip()
    return _extract_last_non_empty_line(output)


def extract_math_prediction(output):
    """Extract a numeric or fraction answer for math problems, including fallback."""
    answers = re.findall(r'^ANSWER:\s*(.*)$', output, re.MULTILINE)
    if answers:
        raw_answer = answers[-1].strip()
        parsed = _handle_math_answer(raw_answer, output)
        return parsed
    return _extract_last_number(output)


def _handle_math_answer(last_answer, full_output):
    """Parse fraction/boxed/numeric answers from the last answer string.
    If no fraction or numeric is found, fallback to the last number in the entire output.
    """
    # A) Check if there's a \boxed{...}.
    box_match = re.findall(r'\\?boxed\s*\{([^}]*)\}', last_answer)
    if box_match:
        return _handle_math_answer(box_match[-1].strip(), full_output)

    # B) Check for latex fraction: \frac{num}{den}.
    fraction_latex = re.findall(r'\\?frac\{\s*(-?[\d\.]+)\s*\}\{\s*([\d\.]+)\s*\}', last_answer)
    if fraction_latex:
        num, den = fraction_latex[-1]
        return f"{num}/{den}"

    # C) Plain fraction: e.g. 20/7.
    fraction_plain = re.findall(r'(-?[\d\.]+\s*/\s*[\d\.]+)', last_answer)
    if fraction_plain:
        return fraction_plain[-1].replace(' ', '')

    # D) Numeric.
    numeric_matches = re.findall(r'(-?[\d,]+\.?\d*)', last_answer)
    if numeric_matches:
        numeric_value = _convert_to_float(numeric_matches[-1])
        return numeric_value

    # E) If no fraction/numeric found, fallback to the last numeric in the entire model output.
    fallback_num = _extract_last_number(full_output)
    return fallback_num


def _convert_to_float(value):
    """Convert string to float if possible, else return original."""
    try:
        clean_value = value.replace(',', '')
        clean_value = re.sub(r'[\\$\{\}]', '', clean_value)
        return float(clean_value)
    except ValueError:
        return value


def _normalize_fact_label(label):
    label_lower = label.lower()
    if "not enough" in label_lower:
        return "NOT ENOUGH INFO"
    if "supports" in label_lower:
        return "SUPPORTS"
    if "refutes" in label_lower:
        return "REFUTES"
    return "NOT ENOUGH INFO"


def _search_label(last_quarter, output):
    for label in ["not enough info", "supports", "refutes"]:
        if label in last_quarter or label in output:
            return _normalize_fact_label(label)
    return "NOT ENOUGH INFO"


def _extract_last_non_empty_line(output):
    lines = [line.strip() for line in output.split('\n') if line.strip()]
    return lines[-1] if lines else output.strip()


def _extract_last_number(output):
    lines = output.split('\n')[-5:]
    for line in reversed(lines):
        numbers = re.findall(r"(-?[\d,]+\.?\d*)", line)
        for num in reversed(numbers):
            converted = _convert_to_float(num)
            if isinstance(converted, float):
                return converted
    return output.strip()


## Evaluation Metrics Functions

Functions to compare predictions with ground truth

In [ ]:
def evaluate_correctness(prediction, ground_truth, dataset_name):
    """Evaluate if the prediction is correct based on dataset type"""

    if dataset_name == "VitaminC" or dataset_name == "FEVER" or dataset_name == "FEVEROUS":
        # Fact verification - direct comparison
        return prediction == ground_truth

    elif dataset_name == "HotpotQA" or dataset_name == "2WikiMultihopQA" or dataset_name == "Bamboogle":
        # QA evaluation - normalize and compare
        return normalize_qa_answers(prediction, ground_truth)

    elif dataset_name == "SVAMP":
        # Math evaluation - numeric comparison
        return evaluate_math_correctness(prediction, ground_truth)

    else:
        # Generic comparison for unknown datasets
        return prediction == ground_truth

def normalize_qa_answers(prediction, ground_truth):
    """Normalize and compare QA answers with flexible matching"""
    if not prediction or not ground_truth:
        return False

    # Handle list or dictionary ground truths
    if isinstance(ground_truth, list):
        ground_truth = " ".join([str(item) for item in ground_truth])
    elif isinstance(ground_truth, dict):
        if "answer" in ground_truth:
            ground_truth = ground_truth["answer"]
        else:
            ground_truth = str(ground_truth)

    # Normalize both strings
    pred_norm = prediction.lower().strip()
    truth_norm = str(ground_truth).lower().strip()

    # Remove punctuation and extra spaces
    pred_norm = re.sub(r'[^\w\s]', '', pred_norm).strip()
    truth_norm = re.sub(r'[^\w\s]', '', truth_norm).strip()

    # Check if prediction contains ground truth or vice versa
    return pred_norm in truth_norm or truth_norm in pred_norm

def evaluate_math_correctness(prediction, ground_truth):
    """Evaluate correctness of math answers with tolerance"""
    try:
        # Convert to numeric values
        if isinstance(prediction, str):
            prediction = float(re.search(r'(-?[\d.]+)', prediction.replace(',', '')).group(1))

        if isinstance(ground_truth, str):
            ground_truth = float(re.search(r'(-?[\d.]+)', ground_truth.replace(',', '')).group(1))

        # Compare with tolerance
        tolerance = 0.01
        return abs(float(prediction) - float(ground_truth)) < tolerance
    except (ValueError, TypeError, AttributeError):
        return False

def get_ground_truth(sample, dataset_name):
    """Extract ground truth from sample based on dataset type"""

    if dataset_name == "VitaminC":
        return sample.get("label", "")

    elif dataset_name == "FEVER" or dataset_name == "FEVEROUS":
        return sample.get("label", "")

    elif dataset_name == "HotpotQA" or dataset_name == "2WikiMultihopQA":
        return sample.get("answer", "")

    elif dataset_name == "SVAMP":
        return sample.get("answer", None)

    elif dataset_name == "Bamboogle":
        return sample.get("answer", "")

    else:
        return None

## Visualization Functions

Functions to create visualizations of results

In [ ]:
def visualize_results(results_df, dataset_name):
    """Create visualizations of results for a specific dataset"""
    output_dir = f"results_{dataset_name}"
    os.makedirs(output_dir, exist_ok=True)

    if 'true_label' in results_df.columns and 'prediction' in results_df.columns:
        # Fact verification datasets
        if len(results_df) > 0:
            try:
                labels = sorted(list(set(results_df['true_label'].unique()).union(set(results_df['prediction'].unique()))))

                # Create confusion matrix
                cm = confusion_matrix(
                    results_df['true_label'],
                    results_df['prediction'],
                    labels=labels
                )

                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
                plt.xlabel('Predicted')
                plt.ylabel('True')
                plt.title(f'Confusion Matrix - {dataset_name}')
                plt.tight_layout()
                plt.savefig(f"{output_dir}/confusion_matrix.png")
                plt.close()

                # Class-wise accuracy
                class_accuracy = {}
                for cls in labels:
                    cls_indices = results_df['true_label'] == cls
                    if cls_indices.any():
                        correct = (results_df.loc[cls_indices, 'prediction'] == cls).sum()
                        class_accuracy[cls] = correct / cls_indices.sum()

                plt.figure(figsize=(10, 6))
                sns.barplot(x=list(class_accuracy.keys()), y=list(class_accuracy.values()))
                plt.ylim(0, 1)
                plt.title(f'Class-wise Accuracy - {dataset_name}')
                plt.ylabel('Accuracy')
                plt.tight_layout()
                plt.savefig(f"{output_dir}/class_accuracy.png")
                plt.close()
            except Exception as e:
                print(f"{RED}Error creating visualizations for {dataset_name}: {e}{ENDC}")

    # Overall accuracy
    plt.figure(figsize=(6, 4))
    accuracy = results_df['correct'].mean()
    plt.bar(['Accuracy'], [accuracy])
    plt.ylim(0, 1)
    plt.title(f'Overall Accuracy - {dataset_name}')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/overall_accuracy.png")
    plt.close()

    # Save sample correct/incorrect examples
    correct_samples = results_df[results_df['correct']].head(3)
    incorrect_samples = results_df[~results_df['correct']].head(3)

    with open(f"{output_dir}/sample_results.txt", 'w') as f:
        f.write(f"=== Sample Correct Examples ({dataset_name}) ===\n\n")
        for _, sample in correct_samples.iterrows():
            f.write(f"Input: {sample.get('input', '')}\n")
            f.write(f"True: {sample.get('true_label', '')}\n")
            f.write(f"Prediction: {sample.get('prediction', '')}\n")
            f.write("\n---\n\n")

        f.write(f"=== Sample Incorrect Examples ({dataset_name}) ===\n\n")
        for _, sample in incorrect_samples.iterrows():
            f.write(f"Input: {sample.get('input', '')}\n")
            f.write(f"True: {sample.get('true_label', '')}\n")
            f.write(f"Prediction: {sample.get('prediction', '')}\n")
            f.write("\n---\n\n")

## Dataset Evaluation Function

Function to evaluate model on each dataset

In [ ]:
def evaluate_dataset(model, tokenizer, data, dataset_name):
    """Evaluate model on a specific dataset"""
    print(f"{BOLD}{CYAN}Starting evaluation of {dataset_name} dataset...{ENDC}")

    results = []
    start_time = time.time()

    for i, sample in enumerate(data):
        # Create appropriate prompt
        prompt = create_prompt(sample, dataset_name)

        # Print the prompt being sent to the model
        print(f"\n{BOLD}{'='*80}{ENDC}")
        print(f"{BLUE}{BOLD}SAMPLE {i+1}/{len(data)} - DATASET: {dataset_name}{ENDC}")
        print(f"{BLUE}{'-'*80}{ENDC}")
        print(f"{GREEN}PROMPT:{ENDC}")
        print(f"{GREEN}{prompt}{ENDC}")
        print(f"{BLUE}{'-'*80}{ENDC}")

        # Run inference
        output = run_inference(model, tokenizer, prompt)

        # Print the model's response
        print(f"{PURPLE}MODEL RESPONSE:{ENDC}")
        print(f"{PURPLE}{output}{ENDC}")

        # Extract prediction and ground truth
        prediction = extract_prediction(output, dataset_name, sample)
        true_label = get_ground_truth(sample, dataset_name)

        # Evaluate correctness
        correct = evaluate_correctness(prediction, true_label, dataset_name)
        correct_color = GREEN if correct else RED

        print(f"{BLUE}{'-'*80}{ENDC}")
        print(f"{YELLOW}PREDICTION: {prediction}{ENDC}")
        print(f"{YELLOW}TRUE LABEL: {true_label}{ENDC}")
        print(f"{correct_color}CORRECT: {correct}{ENDC}")
        print(f"{BOLD}{'='*80}{ENDC}")

        # Store result
        result = {
            "input": prompt,
            "output": output,
            "prediction": prediction,
            "true_label": true_label,
            "correct": correct
        }
        results.append(result)

        # Print progress
        if (i+1) % 10 == 0 or i == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i+1)
            remaining = avg_time * (len(data) - i - 1)
            print(f"{CYAN}Processed {i+1}/{len(data)} samples - "
                  f"Avg time per sample: {avg_time:.2f}s - "
                  f"Estimated time remaining: {remaining/60:.1f} minutes{ENDC}")

    # Calculate overall accuracy
    results_df = pd.DataFrame(results)
    accuracy = results_df['correct'].mean()

    print(f"\n{BOLD}Results for {dataset_name}:{ENDC}")
    print(f"{BOLD}Overall accuracy: {accuracy:.2f}{ENDC}")

    # Save results
    output_dir = f"results_{dataset_name}"
    os.makedirs(output_dir, exist_ok=True)
    results_df.to_csv(f"{output_dir}/results.csv", index=False)

    # Create visualizations
    visualize_results(results_df, dataset_name)

    print(f"{GREEN}Results saved to {output_dir}/{ENDC}")

    return accuracy, results_df

## Model Loading and Dataset Evaluation

Load the model and evaluate it on all datasets

In [ ]:
# Load model and tokenizer
print("Loading model and tokenizer...")
model_path = "/content/drive/Shareddrives/517 nlp project/Models/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")

# Ensure the tokenizer has a pad_token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
    model.resize_token_embeddings(len(tokenizer))

print(f"{GREEN}Model loaded successfully{ENDC}")

Loading model and tokenizer...
Model loaded successfully


In [ ]:
# Define dataset paths
datasets = {
    "VitaminC": "/content/drive/Shareddrives/517 nlp project/data/VitaminC/vitaminc/test.jsonl",
    "2WikiMultihopQA": "/content/drive/Shareddrives/517 nlp project/data/2WikiMultihopQA/test.json",
    "Bamboogle": "/content/drive/Shareddrives/517 nlp project/data/Bamboogle/test.json",
    "FEVER": "/content/drive/Shareddrives/517 nlp project/data/FEVER/fever_test.jsonl",
    "FEVEROUS": "/content/drive/Shareddrives/517 nlp project/data/FEVEROUS/feverous_test.jsonl",
    "HotpotQA": "/content/drive/Shareddrives/517 nlp project/data/HotpotQA/test.json",
    "SVAMP": "/content/drive/Shareddrives/517 nlp project/data/SVAMP/test.json"
}

# Display available datasets
print(f"{CYAN}Available datasets for evaluation:{ENDC}")
for i, dataset_name in enumerate(datasets.keys()):
    print(f"{i+1}. {dataset_name}")

Available datasets for evaluation:
1. VitaminC
2. 2WikiMultihopQA
3. Bamboogle
4. FEVER
5. FEVEROUS
6. HotpotQA
7. SVAMP


Test a single sample for all the datasets to ensure everything is working first

In [ ]:
# Choose which datasets to evaluate
datasets_to_evaluate = [
    "VitaminC",
    "FEVER",
    "FEVEROUS",
    "HotpotQA",
    "2WikiMultihopQA",
    "SVAMP",
    "Bamboogle"
]  # Evaluate all available datasets

# Initialize results summary
summary = {}

# Process each selected dataset
for dataset_name in datasets_to_evaluate:
    if dataset_name not in datasets:
        print(f"{RED}Dataset {dataset_name} not found in available datasets{ENDC}")
        continue

    # Load dataset
    file_path = datasets[dataset_name]
    data = load_dataset(file_path, dataset_name, limit=1)  # Limit to 1 sample

    if data:
        # Evaluate on the dataset
        accuracy, _ = evaluate_dataset(model, tokenizer, data, dataset_name)
        summary[dataset_name] = accuracy

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Loading VitaminC dataset from /content/drive/Shareddrives/517 nlp project/data/VitaminC/vitaminc/test.jsonl...
Loaded 1 samples from VitaminC.
Starting evaluation of VitaminC dataset...

SAMPLE 1/1 - DATASET: VitaminC
--------------------------------------------------------------------------------
PROMPT:
Claim: Westlife made under 23.5 million sales in the UK .

Evidence: According to the British Phonographic Industry ( BPI ) , Westlife has been certified for 13 million albums and 9.8�million singles , with a total of more than 23 million combined sales in the UK .

Think step-by-step to determine if the evidence SUPPORTS, REFUTES, or provides NOT ENOUGH INFO for the claim.

After your thinking, end your answer with one of these:
ANSWER: SUPPORTS or ANSWER: REFUTES or ANSWER: NOT ENOUGH INFO

Always end with the final answer.
--------------------------------------------------------------------------------
MODEL RESPONSE:
Claim: Westlife made under 23.5 million sales in the UK.

Eviden

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Results saved to results_VitaminC/
Loading FEVER dataset from /content/drive/Shareddrives/517 nlp project/data/FEVER/fever_test.jsonl...
Loaded 1 samples from FEVER.
Starting evaluation of FEVER dataset...

SAMPLE 1/1 - DATASET: FEVER
--------------------------------------------------------------------------------
PROMPT:
Claim: Attack on Titan is a person.

Evidence: [[[99632, 112745, 'Attack_on_Titan', 0]], [[99632, 112746, 'Attack_on_Titan', 1]], [[99632, 112747, 'Attack_on_Titan', 2]], [[99632, 112748, 'Attack_on_Titan', 3]], [[99632, 112749, 'Attack_on_Titan', 4]], [[99632, 112750, 'Attack_on_Titan', 7]], [[99632, 112751, 'Attack_on_Titan', 8]], [[99632, 112752, 'Attack_on_Titan', 9]], [[99632, 112753, 'Attack_on_Titan', 10]], [[99632, 112754, 'Attack_on_Titan', 11]], [[99632, 112755, 'Attack_on_Titan', 12]], [[99632, 112756, 'Attack_on_Titan', 13]], [[99632, 112757, 'Attack_on_Titan', 14]], [[99632, 112758, 'Attack_on_Titan', 18]], [[99632, 112759, 'Attack_on_Titan', 19]], [[9963

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Results saved to results_HotpotQA/
Loading 2WikiMultihopQA dataset from /content/drive/Shareddrives/517 nlp project/data/2WikiMultihopQA/test.json...
Loaded 1 samples from 2WikiMultihopQA.
Starting evaluation of 2WikiMultihopQA dataset...

SAMPLE 1/1 - DATASET: 2WikiMultihopQA
--------------------------------------------------------------------------------
PROMPT:


Question: Michael GovanMichael Govan( born 1963) is the director of the Los Angeles County Museum of Art since 2006. Prior to this, Govan worked as the director of the Dia Art Foundation in New York City.
John DonatichJohn Donatich is the Director of Yale University Press.
Peter LevinPeter Levin is an American director of film, television and theatre.
Ian Barry (director)Ian Barry is an Australian director of film and TV.
John Farrell (businessman)John Farrell is the director of YouTube in Latin America.
One Law for the WomanOne Law for the Woman is a 1924 American silent western film directed by Dell Henderson and starring

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Results saved to results_2WikiMultihopQA/
Loading SVAMP dataset from /content/drive/Shareddrives/517 nlp project/data/SVAMP/test.json...
Loaded 1 samples from SVAMP.
Starting evaluation of SVAMP dataset...

SAMPLE 1/1 - DATASET: SVAMP
--------------------------------------------------------------------------------
PROMPT:
Problem: Debby bought 95 soda bottles and 180 water bottles when they were on sale. If she drank 15 water bottles and 54 soda bottles a day
How many days would the water bottles last?

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>

Always end with the final answer.
--------------------------------------------------------------------------------
MODEL RESPONSE:
Problem: Debby bought 95 soda bottles and 180 water bottles when they were on sale. If she drank 15 water bottles and 54 soda bottles a day
How many days would the water bottles last?

Think step-by-step to solve

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


MODEL RESPONSE:


Question: Who was president of the United States in the year that Citibank was founded?

Think step-by-step to answer the question based on the context.

After your thinking, provide your answer in this exact format:
ANSWER: <finalAnswer>

Always end with the final answer. Do not forget to follow the format to the letter.


Since the question is about the founding of Citibank and the presidency of the United States, I will provide the answer in the required format.


ANSWER: William Howard Taft
--------------------------------------------------------------------------------
PREDICTION: William Howard Taft
TRUE LABEL: james madison
CORRECT: False
Processed 1/1 samples - Avg time per sample: 1.47s - Estimated time remaining: 0.0 minutes

Results for Bamboogle:
Overall accuracy: 0.00
Results saved to results_Bamboogle/


In [ ]:
# Choose which datasets to evaluate
# You can modify this list to evaluate specific datasets
# datasets_to_evaluate = ["VitaminC", "FEVER"]  # Uncomment to evaluate specific datasets
datasets_to_evaluate = [
    # "VitaminC",
    # "FEVER",
    # "FEVEROUS",
    # "HotpotQA",
    # "2WikiMultihopQA",
    "SVAMP",
    # "Bamboogle"
]  # Evaluate all available datasets
# Initialize results summary
summary = {}

# Process each selected dataset
for dataset_name in datasets_to_evaluate:
    if dataset_name not in datasets:
        print(f"{RED}Dataset {dataset_name} not found in available datasets{ENDC}")
        continue

    # Load dataset
    file_path = datasets[dataset_name]
    data = load_dataset(file_path, dataset_name, limit=150)  # Limit to 150 samples

    if data:
        # Evaluate on the dataset
        accuracy, _ = evaluate_dataset(model, tokenizer, data, dataset_name)
        summary[dataset_name] = accuracy

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Loading SVAMP dataset from /content/drive/Shareddrives/517 nlp project/data/SVAMP/test.json...
Loaded 150 samples from SVAMP.
Starting evaluation of SVAMP dataset...

SAMPLE 1/150 - DATASET: SVAMP
--------------------------------------------------------------------------------
PROMPT:
Problem: Debby bought 95 soda bottles and 180 water bottles when they were on sale. If she drank 15 water bottles and 54 soda bottles a day
How many days would the water bottles last?

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>
Never end your final answer with a latex boxed format.
Always end with the final answer.
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


MODEL RESPONSE:
Problem: Debby bought 95 soda bottles and 180 water bottles when they were on sale. If she drank 15 water bottles and 54 soda bottles a day
How many days would the water bottles last?

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>
Never end your final answer with a latex boxed format.
Always end with the final answer. Don't forget to include the boxed answer.

## Step 1: Calculate the total number of soda bottles Debby bought.
Debby bought 95 soda bottles.

## Step 2: Calculate the total number of water bottles Debby bought.
Debby bought 180 water bottles.

## Step 3: Calculate the total number of soda bottles Debby drank.
Debby drank 54 soda bottles.

## Step 4: Calculate the number of water bottles Debby drank.
Debby drank 15 water bottles.

## Step 5: Calculate the number of water bottles left.
Water bottles left = Total water bottles - Water bottles drank = 180 - 15 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

MODEL RESPONSE:
Problem: He then went to see the oranges being harvested. He found out that they harvest 8 sacks per day.
How many days will it take to harvest 24 sacks of oranges?

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>
Never end your final answer with a latex boxed format.
Always end with the final answer. I will do this for you.

## Step 1: Determine the number of sacks harvested per day
The oranges harvest 8 sacks per day.

## Step 2: Divide the total number of sacks by the number of sacks harvested per day
To find out how many days it will take to harvest 24 sacks, we need to divide 24 by 8.

## Step 3: Calculate the number of days
24 / 8 = 3

The final answer is: $\boxed{3}$
--------------------------------------------------------------------------------
PREDICTION: 3.0
TRUE LABEL: 3.0
CORRECT: True

SAMPLE 148/150 - DATASET: SVAMP
------------------------------------------

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


MODEL RESPONSE:
Problem: David did 40 more push-ups but 17 less crunches than Zachary in gym class today. If Zachary did 34 push-ups and 62 crunches
How many crunches did David do?

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>
Never end your final answer with a latex boxed format.
Always end with the final answer. The final answer is 45.

Step 1:  Identify the information given in the problem.
David did 40 more push-ups than Zachary. Zachary did 34 push-ups. David did 17 less crunches than Zachary. Zachary did 62 crunches.

Step 2:  Calculate the number of push-ups David did.
David did 40 more push-ups than Zachary. Zachary did 34 push-ups. So David did 34 + 40 = 74 push-ups.

Step 3:  Calculate the number of crunches David did.
David did 17 less crunches than Zachary. Zachary did 62 crunches. So David did 62 - 17 = 45 crunches.

ANSWER: 45
---------------------------------------------

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


MODEL RESPONSE:
Problem: The Razorback t-shirt shop made $ 51 dollars from selling 3 t-shirt during the Arkansas and Texas tech game they.
What is the cost of each t-shirt?

Think step-by-step to solve this math problem.

After your thinking, provide your final numeric answer in this exact format:
ANSWER: <finalAnswer>
Never end your final answer with a latex boxed format.
Always end with the final answer. I will do this for you.

## Step 1: Identify the given information
The Razorback t-shirt shop made $51 from selling 3 t-shirts during the Arkansas and Texas tech game.

## Step 2: Determine the cost per t-shirt
To find the cost of each t-shirt, we need to divide the total amount made ($51) by the number of t-shirts sold (3).

## Step 3: Calculate the cost per t-shirt
Cost per t-shirt = Total amount made / Number of t-shirts sold
Cost per t-shirt = $51 / 3
Cost per t-shirt = $17

The final answer is: $\boxed{17}$
------------------------------------------------------------------------

## Results Visualization

Create summary visualizations of model performance across datasets

In [ ]:
# Print summary of results
print(f"\n{BOLD}{CYAN}Summary of Results:{ENDC}")
for dataset, accuracy in summary.items():
    print(f"{dataset}: {accuracy:.4f}")

# Create summary visualization
if summary:
    plt.figure(figsize=(12, 6))
    datasets = list(summary.keys())
    accuracies = list(summary.values())

    # Sort by accuracy
    sorted_indices = np.argsort(accuracies)[::-1]
    sorted_datasets = [datasets[i] for i in sorted_indices]
    sorted_accuracies = [accuracies[i] for i in sorted_indices]

    bars = plt.bar(sorted_datasets, sorted_accuracies)

    # Add value labels on top of bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.4f}', ha='center', va='bottom')

    plt.ylim(0, 1.1)  # Set y limit to 0-1 with a small margin
    plt.xlabel('Datasets')
    plt.ylabel('Accuracy')
    plt.title('Llama-3.2-1B Performance Across Datasets')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("summary_results.png")
    plt.show()

    # Save summary to CSV
    summary_df = pd.DataFrame(list(summary.items()), columns=['Dataset', 'Accuracy'])
    summary_df = summary_df.sort_values('Accuracy', ascending=False)
    summary_df.to_csv("summary_results.csv", index=False)

    print(f"\n{GREEN}Summary saved to summary_results.csv and summary_results.png{ENDC}")
else:
    print(f"\n{RED}No results to visualize{ENDC}")